Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# How to use Function Calling with Gemini

This notebook outlines how to interact with Vertex AI's Gemini models to call external API's using Function Calling. More info can be found at https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [ ]:
project_id = "YOUR_PROJECT_ID"
location = "global"
region = "us-central1"

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [ ]:
#pip install -r requirements.txt

Now we will import all required modules. For our purpose, we will be utilizing the following:

- requests - This module will allow us to interact directly with external REST API's. 
- FunctionDeclaration - Used to define the function to be called by the model

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part, Tool, ChatSession, FunctionDeclaration
import vertexai.preview.generative_models as generative_models
from vertexai.preview.generative_models import grounding
import requests

## Define the Gemini Function

Next we will define a function to be called by the gemini model. This function will be used to determine the current time based on a specified location provided by the user.

In [ ]:
get_time_func = FunctionDeclaration(
    name="get_time_func",
    description="Get the current time in a given location. Include the area and location, for example 'America/New_York', 'Asia/Dubai' and 'Africa/Cairo'",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

time_tool = Tool(
    function_declarations=[get_time_func],
)

Initialize the mode, specifying the "time_tool" variable linked to the get_time_func function

In [ ]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[time_tool],
)
chat = model.start_chat()

## Submit a prompt, call the function and return the response

In this example, we're using the external worldtimeapi.org api to find the current time in a specific timezone based on a specified area. The supported areas can be listed using running the following command 

In [ ]:
response = requests.get('http://worldtimeapi.org/api/timezone')
supported_areas = response.json()

for area in supported_areas:
    print(area)

Define a request to get the current time in Chicago

In [ ]:
prompt = "What time is it in Chicago?"

Submit the prompt and print the response

In [ ]:
response = chat.send_message(prompt)

#--- Uncomment to see the full response structure
print(response.candidates[0].content.parts[0])

#-- If the response includes the "function_call" attribute, capture the function name, call the external API, and return the response.
if response.candidates[0].content.parts[0].function_call:
    function_name = response.candidates[0].content.parts[0].function_call.name
    
    params = {}
    for key, value in response.candidates[0].content.parts[0].function_call.args.items():
        params[key] = value
    params
    
    #url = f"https://worldtimeapi.org/api/timezone/america/{params['location']}"
    url = f"https://worldtimeapi.org/api/timezone/{params['location']}"
    
    api_response = requests.get(url, params=params)
    api_response.text
    
    response = chat.send_message(
        Part.from_function_response(
            name="get_time_func",
            response={
                "content": api_response.text,
            },
        ),
    )


In [ ]:
#--- Print the text section of the response which includes the current time.
print(response.text)